SageMaker Studio notebooks provide a set of built-in images for popular data science and ML frameworks and compute options to run notebooks. The built-in SageMaker images contain the Amazon SageMaker Python SDK and the latest version of the backend runtime process, also called kernel. With the custom images feature, you can register custom built images and kernels, and make them available to all users sharing a SageMaker Studio domain. You can start by cloning and extending one of the example Docker files provided by SageMaker, or build your own images from scratch.


You can add a custom **R** image to SageMaker Studio so you can build and train your R models with SageMaker. After attaching the custom R image, you can select the image in Studio and use R to access the SDKs using the **RStudio reticulate** package.

You can create images and image versions and attach image versions to your domain using the SageMaker Studio Control Panel, the AWS SDK for Python (Boto3), and the AWS Command Line Interface (AWS CLI).

In [ ]:
import sagemaker

In [ ]:
ROLE_ARN = sagemaker.get_execution_role()
ROLE_ARN

In [ ]:
# !aws sagemaker describe-notebook-instance --notebook-instance-name 

In [ ]:
!pwd

In [ ]:
%%sh

ACCOUNT_ID=$(aws sts get-caller-identity --query Account --output text)
# Get the region defined in the current configuration (default to us-west-2 if none defined)
REGION=$(aws configure list | grep region | awk '{print $2}')
algorithm_name=smstudio-custom
fullname=${ACCOUNT_ID}.dkr.ecr.${REGION}.amazonaws.com/${algorithm_name}:latest




# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# login to ECR repository
aws ecr --region ${REGION} get-login-password | docker login --username AWS \
    --password-stdin fullname

# Build the docker image and push to Amazon ECR (modify image tags and name as required)
$(aws ecr get-login --region ${REGION} --no-include-email)


# with the full name.
docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}


# Using with SageMaker Studio
## Create SageMaker Image with the image in ECR (modify image name as required)
ROLE_ARN="arn:aws:iam::367158743199:role/MySagemakerRole"
echo ${ROLE_ARN}

aws sagemaker create-image \
    --region ${REGION} \
    --image-name r-image \
    --role-arn ${ROLE_ARN}
          
    
aws sagemaker create-image-version \
    --image-name r-image \
    --base-image $fullname

## Create AppImageConfig for this image (modify AppImageConfigName and 
## KernelSpecs in app-image-config-input.json as needed)
## note that 'file://' is required in the file path
aws sagemaker create-app-image-config \
    --region ${REGION} \
    --cli-input-json file://app-image-config-input.json
    